# 实时疫情数据

数据来源：腾讯新闻

## 数据获取

In [3]:
import requests
import json
import pandas as pd

url = r'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
response = requests.get(url)
result = json.loads(response.json()['data'])
data_china = result['areaTree'][0]['children']

# item['total']总数据，替换为item['today']可获得当日新增数据
def GetTotalChina(data_china):
    province_list=[]
    city_list=[]
    confirm_list=[]
    dead_list=[]
    suspect_list=[]
    heal_list=[]
    
    for each in data_china:
        for item in each['children']:
            province_list.append(each['name'])
            city_list.append(item['name'])
            confirm_list.append(item['total']['confirm'])
            suspect_list.append(item['total']['suspect'])
            dead_list.append(item['total']['dead'])
            heal_list.append(item['total']['heal'])

    ddict = {'province':province_list,
            'city':city_list,
            'confirm':confirm_list,
            'suspect':suspect_list,
            'dead':dead_list,
            'heal':heal_list}
    df = pd.DataFrame(ddict, columns=['province','city','confirm','suspect','dead', 'heal'])
    confirm_prov = df.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)
    
    return df

def GetTodayChina(data_china):
    province_list=[]
    city_list=[]
    confirm_list=[]
    dead_list=[]
    suspect_list=[]
    heal_list=[]
    
    for each in data_china:
        for item in each['children']:
            province_list.append(each['name'])
            city_list.append(item['name'])
            confirm_list.append(item['today']['confirm'])
            suspect_list.append(item['today']['suspect'])
            dead_list.append(item['today']['dead'])
            heal_list.append(item['today']['heal'])

    ddict = {'province':province_list,
            'city':city_list,
            'confirm':confirm_list,
            'suspect':suspect_list,
            'dead':dead_list,
            'heal':heal_list}
    df = pd.DataFrame(ddict, columns=['province','city','confirm','suspect','dead', 'heal'])
    confirm_prov = df.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)
    
    return df

dftotal = GetTotalChina(data_china)

## 优化数据获取

以下代码参考：https://github.com/ni1o1/pygeo-tutorial/blob/master/pyecharts_tutorial.ipynb

In [4]:
import requests
import json
import pandas as pd

url = r'https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5'
response = requests.get(url)
result = json.loads(response.json()['data'])
datachina = pd.DataFrame(result['areaTree'][0]['children'])
dataprovince = pd.DataFrame(datachina['total'].apply(lambda r:[r]).sum())
# dataprovince = list(datachina['total'])
dataprovince['province'] = datachina['name']

In [5]:
dataprovince[['province','confirm']].values

array([['湖北', 16678],
       ['浙江', 895],
       ['广东', 895],
       ['河南', 764],
       ['湖南', 661],
       ['江西', 548],
       ['安徽', 530],
       ['重庆', 376],
       ['江苏', 341],
       ['山东', 307],
       ['四川', 301],
       ['北京', 253],
       ['上海', 243],
       ['福建', 205],
       ['黑龙江', 190],
       ['陕西', 165],
       ['广西', 150],
       ['河北', 135],
       ['云南', 124],
       ['海南', 91],
       ['辽宁', 81],
       ['山西', 81],
       ['天津', 69],
       ['贵州', 64],
       ['甘肃', 57],
       ['吉林', 54],
       ['内蒙古', 42],
       ['宁夏', 34],
       ['新疆', 32],
       ['香港', 21],
       ['青海', 17],
       ['台湾', 11],
       ['澳门', 10],
       ['西藏', 1]], dtype=object)

# 绘制确诊病例数分布图（直辖市、省、自治区级）

## 按原数据绘图

In [6]:
from pyecharts import options as opts
from pyecharts.charts import Map

def DrawMapChina(data):
    
    confirm_prov = data.groupby(['province'])['confirm'].sum().reset_index()
    confirm_prov = confirm_prov.sort_values(by=['confirm'], ascending=False)
    confirm_prov = confirm_prov.reset_index(drop=True)

    province = confirm_prov['province'].values.tolist()
    confirm = confirm_prov['confirm'].values.tolist()

    geo = Map()
    geo.add("2019-nCoV确诊人数", [list(z) for z in zip(province, confirm)], "china", is_roam=False, zoom=1.2)
    geo.set_global_opts(visualmap_opts=opts.VisualMapOpts(max_=300),
                       title_opts=opts.TitleOpts(title="2019-nCoV"))
    geo.render('map.html')
    return geo.render_notebook()
    
DrawMapChina(dftotal)

## 按优化获得的数据绘图

In [7]:
from pyecharts import options as opts
from pyecharts.charts import Map
import datetime
date = datetime.datetime.now().strftime('%Y-%m-%d')
c = (
    Map()
    .add("确诊病例数",
        dataprovince[['province','confirm']].values,
        "china",
        is_roam=False,
        zoom=1.2,
        is_map_symbol_show=False,
        label_opts=opts.LabelOpts(position='inside'),
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(title='疫情地图 '+date),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,
                                         pieces=[{'min':10000,'label':'10000人以上'},
                                                 {'min':1000,'max':9999, 'label':'1000-9999人'},
                                                 {'min':500,'max':999,'label':'500-999人'},
                                                 {'min':100,'max':499,'label':'100-499人'},
                                                 {'min':10,'max':99,'label':'10-99人'},
                                                 {'min':1,'max':9,'label':'1-9人'}],
                                         range_color=["#ffc6b3","#ff8c66","#ff9980","#e60000","#cc3300","#b30000","#800000"]
                                         ),
                    )
)
c.render("疫情地图.html")
# c.render_notebook()

'E:\\pythonx\\疫情地图.html'

![1sNvjK.png](https://s2.ax1x.com/2020/02/05/1sNvjK.png)

# 绘制确诊病例热力图（直辖市、省、自治区级）

In [8]:
from pyecharts import options as opts
from pyecharts.charts import Geo
from pyecharts.globals import ChartType, SymbolType
import datetime

date = datetime.datetime.now().strftime('%Y-%m-%d')

c = (
    Geo()
    .add_schema(maptype="china",
               is_roam=False,
               zoom=1,)
    .add(
        "确诊人数",
        dataprovince[['province','confirm']].values,
        type_=ChartType.HEATMAP,
    )
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(
        visualmap_opts=opts.VisualMapOpts(max_=200),
        title_opts=opts.TitleOpts(title="HeatMap "+date),
    )
)
c.render('疫情热力图.html')
# c.render_notebook()

'E:\\pythonx\\疫情热力图.html'

![1sNznO.png](https://s2.ax1x.com/2020/02/05/1sNznO.png)